In [28]:
import os
import pickle
from pathlib import Path
from typing import Generator

import numpy as np
import pandas as pd
import scikit_posthocs as sp
from IPython.display import HTML, display
from scipy import stats
from yaml import safe_load
from jmetal.lab.statistical_test.functions import (
    friedman_aligned_rank_test,
    friedman_aligned_ph_test,
)


pd.set_option("display.max_columns", 100)


def friedmann_nemenyi_test(
    unravelled_detailed_results: dict[str : dict[tuple[str, str], list[float]]],
) -> None:
    for metric in unravelled_detailed_results.keys():
        metric_df = pd.DataFrame(unravelled_detailed_results[metric])
        metric_vals = [metric_df[col].values for col in metric_df.columns]
        out = stats.friedmanchisquare(*metric_vals)
        print(
            f"Friedman Test {metric}: statistic={out.statistic:.3f}, pvalue={out.pvalue:.3f}"
        )
        metric_vals = np.array(metric_vals).T
        out = sp.posthoc_nemenyi_friedman(metric_vals)
        print("Nemenyi post-hoc test")
        feature_names = metric_df.columns
        out.index = feature_names
        out.columns = feature_names
        out = out.sort_index(axis=0).sort_index(axis=1)
        display(out)


def aligned_friedmann_holm_test(
    unravelled_detailed_results: dict[str : dict[tuple[str, str], list[float]]],
) -> None:
    for metric in unravelled_detailed_results.keys():
        metric_df = pd.DataFrame(unravelled_detailed_results[metric])
        # metric_vals = np.asarray([metric_df[col].values for col in metric_df.columns])
        metric_vals = metric_df.values

        out = friedman_aligned_rank_test(metric_vals)
        chi2_stat = out.loc['Aligned Rank stat'].iloc[0]
        p_value = out.loc['p-value'].iloc[0]
        print(f"Aligned-rank Friedman χ² {metric} = {chi2_stat:.3f}, p = {p_value:.3f}")

        z_vals, out, _ = friedman_aligned_ph_test(
            metric_vals, apv_procedure="Holm"  # Holm step-down correction
        )
        print("Holm post-hoc test")
        feature_names = metric_df.columns
        out.index = feature_names
        out.columns = feature_names
        out = out.sort_index(axis=0).sort_index(axis=1)
        display(out)


def present_results(
    paths: Generator,
    val_method: str = "lopo",
    remove_xgboost: bool = False,
    remove_chronos_small_from_test: bool = False,
    which_test: str = "friedmann-nemenyi",
    test_args: dict = {},
) -> None:
    results = []
    for reports_path in paths:
        report = pd.read_csv(reports_path, index_col=0)
        conf = safe_load(open(reports_path.parent / ".hydra/config.yaml"))
        if conf["validation_method"]["_target_"].split(".")[-1].lower() != val_method:
            continue
        model_name: str = conf["model"]["model"]["_target_"].split(".")[-1]
        features_name = (
            conf["feature_extractor"]["_target_"].split(".")[-1]
            if "model_name" not in conf["feature_extractor"]
            else conf["feature_extractor"]["model_name"]
        )
        validation_method = conf["validation_method"]["_target_"].split(".")[-1]
        if "aggregator" not in conf:
            aggregator = "MeanTimeAggregator"
        else:
            aggregator = (
                conf["aggregator"]["_target_"].split(".")[-1]
                if "_target_" in conf["aggregator"]
                else None
            )
        report_results = {}
        for col in report.columns:
            report_results[f"{col} avg"] = report[col].mean()
            report_results[f"{col} sem"] = report[col].sem() * 1.98  # 95% CI

        dataset = conf["dataset"]
        side = conf["side"]
        resampling = (
            conf["resampling"]["_target_"].split(".")[-1]
            if "resampling" in conf
            else "None"
        )
        resampling = resampling if resampling != "NoUnderSampler" else "None"
        # Collect results in a list of dicts
        if remove_xgboost and model_name == "XGBClassifier":
            continue
        results.append(
            {
                "Dataset": dataset,
                "Side": side,
                "Model": model_name,
                "Resampling": resampling,
                "Features": features_name,
                "Aggregator": aggregator,
                "Validation": validation_method,
                "Detailed Report": report,
                **report_results,
            }
        )

    # After the loop, display as a table
    df_results = pd.DataFrame(results)
    for (dataset, side, resampling), group in df_results.groupby(
        ["Dataset", "Side", "Resampling"]
    ):
        display(
            HTML(
                f"""
            <div style='background-color:#ffe6e6; padding:18px; margin:10px 0; border-radius:8px;'>
                <h2 style='color:#b30000; margin:0; font-size:2em;'>
                Results for Dataset: <i>{dataset}</i>, Side: <i>{side}</i>, resampling: <i>{resampling}</i>
                </h2>
            </div>
            """
            )
        )
        display(
            group.sort_values(by=["Model", "Features", "Aggregator"])
            .drop(columns=["Detailed Report"])
            .drop_duplicates()
        )
        if "aggregator" in test_args.keys() and group["Aggregator"].nunique() > 2:
            group = group[
                (group["Aggregator"] == test_args["aggregator"])
                | (group["Aggregator"].isnull())
            ]

        unravelled_detailed_results = {
            metric: {} for metric in group["Detailed Report"].iloc[0].columns
        }
        for i, model_results in group.iterrows():
            for metric in model_results["Detailed Report"].columns:
                cv_results = model_results["Detailed Report"][metric]
                unravelled_detailed_results[metric][
                    (model_results["Model"], model_results["Features"])
                ] = cv_results

        if which_test == "friedmann-nemenyi":
            friedmann_nemenyi_test(unravelled_detailed_results)
        elif which_test == "alignedfriedmann-holm":
            aligned_friedmann_holm_test(unravelled_detailed_results)
        else:
            raise ValueError(f"Unknown test: {which_test}")

    return results

In [30]:
results_path: str = "../outputs/"
all_results = list(Path(results_path).glob("*/*/*/reports.csv"))
results_path_adula: str = "../outputs_adula/"
all_results += list(Path(results_path_adula).glob("*/*/*/reports.csv"))


results_lopo = present_results(
    all_results,
    val_method="lopo",
    remove_xgboost=True,
    remove_chronos_small_from_test=True,
    which_test="friedmann-nemenyi",
    test_args={
        "aggregator": "MeanChanAggregator"}
)

,Dataset,Side,Model,Resampling,Features,Aggregator,Validation,accuracy_score avg,accuracy_score sem,balanced_accuracy_score avg,balanced_accuracy_score sem,matthews_corrcoef avg,matthews_corrcoef sem,f1_score avg,f1_score sem,precision_score avg,precision_score sem,recall_score avg,recall_score sem,roc_auc_score avg,roc_auc_score sem
4,apsync,engagement_10sec,DummyClassifier,None,HandcraftedFeatureExtractor,None,LOPO,0.421389,0.092253,0.518713,0.027086,0.034524,0.051993,0.318314,0.174510,0.272775,0.169406,0.457165,0.268180,0.518713,0.027086
6,apsync,engagement_10sec,LogisticRegression,None,AutonLab/MOMENT-1-large,MeanChanAggregator,LOPO,0.484316,0.153246,0.500000,0.000000,0.000000,0.000000,0.065514,0.129718,0.042503,0.084156,0.142857,0.282857,0.500000,0.000000
3,apsync,engagement_10sec,LogisticRegression,None,HandcraftedFeatureExtractor,None,LOPO,0.627536,0.107493,0.641607,0.092030,0.282201,0.157940,0.535047,0.163484,0.623977,0.168456,0.580068,0.249043,0.641607,0.092030
2,apsync,engagement_10sec,LogisticRegression,None,amazon/chronos-t5-large,MeanChanAggregator,LOPO,0.482812,0.153343,0.498413,0.003143,-0.015532,0.030753,0.065514,0.129718,0.042503,0.084156,0.142857,0.282857,0.498413,0.003143
5,apsync,engagement_10sec,LogisticRegression,None,amazon/chronos-t5-small,MeanChanAggregator,LOPO,0.499654,0.086121,0.500339,0.050637,-0.003384,0.100121,0.297661,0.185441,0.311768,0.213402,0.328211,0.214678,0.500339,0.050637
1,apsync,engagement_10sec,LogisticRegression,None,ibm-granite/granite-timeseries-patchtsmixer,MeanChanAggregator,LOPO,0.425518,0.088017,0.492814,0.032105,-0.009583,0.084659,0.288127,0.098984,0.466334,0.196568,0.360991,0.234210,0.492814,0.032105


Friedman Test accuracy_score: statistic=10.223, pvalue=0.069
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.939262   
                   HandcraftedFeatureExtractor                                    0.123979   
                   amazon/chronos-t5-large                                        0.970025   
                   amazon/chronos-t5-small                                        0.522199   
                   ibm-granite/granite-timeseries-patchtsmixer                    1.000000   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.939262   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.617519   
                   amazon/chronos-t5-large                                    0.999992   
                   amazon/chronos-t5-small                                    0.970025   
                   ibm-granite/granite-timeseries-patchtsmixer                0.956477   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.123979   
LogisticRegression AutonLab/MOMENT-1-large                                        0.617519   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        0.522199   
                   amazon/chronos-t5-small                                        0.970025   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.146350   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.970025   
LogisticRegression AutonLab/MOMENT-1-large                                    0.999992   
                   HandcraftedFeatureExtractor                                0.522199   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.939262   
                   ibm-granite/granite-timeseries-patchtsmixer                0.980292   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.522199   
LogisticRegression AutonLab/MOMENT-1-large                                    0.970025   
                   HandcraftedFeatureExtractor                                0.970025   
                   amazon/chronos-t5-large                                    0.939262   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.569924   

                                                                                                            
                                                               ibm-granite/granite-timeseries-patchtsmixer  
DummyClassifier    HandcraftedFeatureExtractor                                                    1.000000  
LogisticRegression AutonLab/MOMENT-1-large                                     

Friedman Test balanced_accuracy_score: statistic=12.557, pvalue=0.028
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.998171   
                   HandcraftedFeatureExtractor                                    0.171582   
                   amazon/chronos-t5-large                                        0.992869   
                   amazon/chronos-t5-small                                        0.999745   
                   ibm-granite/granite-timeseries-patchtsmixer                    1.000000   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.998171   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.059732   
                   amazon/chronos-t5-large                                    0.999992   
                   amazon/chronos-t5-small                                    0.980292   
                   ibm-granite/granite-timeseries-patchtsmixer                0.996187   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.171582   
LogisticRegression AutonLab/MOMENT-1-large                                        0.059732   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        0.039878   
                   amazon/chronos-t5-small                                        0.302328   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.199780   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.992869   
LogisticRegression AutonLab/MOMENT-1-large                                    0.999992   
                   HandcraftedFeatureExtractor                                0.039878   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.956477   
                   ibm-granite/granite-timeseries-patchtsmixer                0.987740   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.999745   
LogisticRegression AutonLab/MOMENT-1-large                                    0.980292   
                   HandcraftedFeatureExtractor                                0.302328   
                   amazon/chronos-t5-large                                    0.956477   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.999938   

                                                                                                            
                                                               ibm-granite/granite-timeseries-patchtsmixer  
DummyClassifier    HandcraftedFeatureExtractor                                                    1.000000  
LogisticRegression AutonLab/MOMENT-1-large                                     

...

DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.084143   
                   HandcraftedFeatureExtractor                                    0.000200   
                   amazon/chronos-t5-large                                        0.000077   
                   amazon/chronos-t5-small                                        0.000317   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.000106   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.084143   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.537957   
                   amazon/chronos-t5-large                                    0.403474   
                   amazon/chronos-t5-small                                    0.607089   
                   ibm-granite/granite-timeseries-patchtsmixer                0.447113   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.000200   
LogisticRegression AutonLab/MOMENT-1-large                                        0.537957   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        0.999948   
                   amazon/chronos-t5-small                                        0.999998   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.999993   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.000077   
LogisticRegression AutonLab/MOMENT-1-large                                    0.403474   
                   HandcraftedFeatureExtractor                                0.999948   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.999617   
                   ibm-granite/granite-timeseries-patchtsmixer                1.000000   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.000317   
LogisticRegression AutonLab/MOMENT-1-large                                    0.607089   
                   HandcraftedFeatureExtractor                                0.999998   
                   amazon/chronos-t5-large                                    0.999617   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.999889   

                                                                                                            
                                                               ibm-granite/granite-timeseries-patchtsmixer  
DummyClassifier    HandcraftedFeatureExtractor                                                    0.000106  
LogisticRegression AutonLab/MOMENT-1-large                                     

Friedman Test recall_score: statistic=51.116, pvalue=0.000
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.962412   
                   HandcraftedFeatureExtractor                                    0.000034   
                   amazon/chronos-t5-large                                        0.128677   
                   amazon/chronos-t5-small                                        0.150842   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.001784   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                            9.624121e-01   
LogisticRegression AutonLab/MOMENT-1-large                                1.000000e+00   
                   HandcraftedFeatureExtractor                            4.147301e-07   
                   amazon/chronos-t5-large                                1.191366e-02   
                   amazon/chronos-t5-small                                1.499661e-02   
                   ibm-granite/granite-timeseries-patchtsmixer            4.694174e-05   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                3.357440e-05   
LogisticRegression AutonLab/MOMENT-1-large                                    4.147301e-07   
                   HandcraftedFeatureExtractor                                1.000000e+00   
                   amazon/chronos-t5-large                                    2.182088e-01   
                   amazon/chronos-t5-small                                    1.891656e-01   
                   ibm-granite/granite-timeseries-patchtsmixer                9.473232e-01   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.128677   
LogisticRegression AutonLab/MOMENT-1-large                                    0.011914   
                   HandcraftedFeatureExtractor                                0.218209   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.759397   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.150842   
LogisticRegression AutonLab/MOMENT-1-large                                    0.014997   
                   HandcraftedFeatureExtractor                                0.189166   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.718178   

                                                                                                            
                                                               ibm-granite/granite-timeseries-patchtsmixer  
DummyClassifier    HandcraftedFeatureExtractor                                                    0.001784  
LogisticRegression AutonLab/MOMENT-1-large                                     

Friedman Test roc_auc_score: statistic=29.114, pvalue=0.000
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.779036   
                   HandcraftedFeatureExtractor                                    0.000665   
                   amazon/chronos-t5-large                                        0.007396   
                   amazon/chronos-t5-small                                        0.003048   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.001784   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.779036   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.064018   
                   amazon/chronos-t5-large                                    0.267036   
                   amazon/chronos-t5-small                                    0.162927   
                   ibm-granite/granite-timeseries-patchtsmixer                0.118566   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.000665   
LogisticRegression AutonLab/MOMENT-1-large                                        0.064018   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        0.989528   
                   amazon/chronos-t5-small                                        0.998982   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.999889   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.007396   
LogisticRegression AutonLab/MOMENT-1-large                                    0.267036   
                   HandcraftedFeatureExtractor                                0.989528   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.999889   
                   ibm-granite/granite-timeseries-patchtsmixer                0.998982   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.003048   
LogisticRegression AutonLab/MOMENT-1-large                                    0.162927   
                   HandcraftedFeatureExtractor                                0.998982   
                   amazon/chronos-t5-large                                    0.999889   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.999993   

                                                                                                            
                                                               ibm-granite/granite-timeseries-patchtsmixer  
DummyClassifier    HandcraftedFeatureExtractor                                                    0.001784  
LogisticRegression AutonLab/MOMENT-1-large                                     

In [30]:
results_lopo = present_results(
    all_results,
    val_method="tacv",
    remove_xgboost=True,
    remove_chronos_small_from_test=True,
)

,Dataset,Side,Model,Resampling,Features,Aggregator,Validation,accuracy_score avg,accuracy_score sem,balanced_accuracy_score avg,balanced_accuracy_score sem,matthews_corrcoef avg,matthews_corrcoef sem,f1_score avg,f1_score sem,precision_score avg,precision_score sem,recall_score avg,recall_score sem,roc_auc_score avg,roc_auc_score sem
4,apsync,engagement_10sec,DummyClassifier,None,HandcraftedFeatureExtractor,None,TACV,0.478237,0.051894,0.500512,0.031328,0.001778,0.061588,0.387970,0.202726,0.357265,0.200925,0.443405,0.221716,0.500512,0.031328
6,apsync,engagement_10sec,LogisticRegression,None,AutonLab/MOMENT-1-large,MeanChanAggregator,TACV,0.534116,0.090357,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000
3,apsync,engagement_10sec,LogisticRegression,None,HandcraftedFeatureExtractor,None,TACV,0.640500,0.131275,0.615714,0.136109,0.241794,0.286743,0.462302,0.302731,0.625714,0.362842,0.466578,0.334841,0.615714,0.136109
2,apsync,engagement_10sec,LogisticRegression,None,amazon/chronos-t5-large,MeanChanAggregator,TACV,0.534116,0.090357,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000
5,apsync,engagement_10sec,LogisticRegression,None,amazon/chronos-t5-small,MeanChanAggregator,TACV,0.508916,0.058305,0.523843,0.049311,0.041462,0.110617,0.297648,0.210281,0.364110,0.201832,0.312121,0.266114,0.523843,0.049311
1,apsync,engagement_10sec,LogisticRegression,None,ibm-granite/granite-timeseries-patchtsmixer,MeanChanAggregator,TACV,0.504563,0.053762,0.519786,0.050948,0.021321,0.133771,0.361768,0.191620,0.405507,0.242627,0.368939,0.210530,0.519786,0.050948


Friedman Test: statistic=21.023, pvalue=0.000
Nemenyi post-hoc test


,"(DummyClassifier, HandcraftedFeatureExtractor)","(LogisticRegression, AutonLab/MOMENT-1-large)","(LogisticRegression, HandcraftedFeatureExtractor)","(LogisticRegression, amazon/chronos-t5-large)","(LogisticRegression, ibm-granite/granite-timeseries-patchtsmixer)"
"(DummyClassifier, HandcraftedFeatureExtractor)",1.000000,0.603784,0.124583,0.603784,0.986698
"(LogisticRegression, AutonLab/MOMENT-1-large)",0.603784,1.000000,0.001346,1.000000,0.294070
"(LogisticRegression, HandcraftedFeatureExtractor)",0.124583,0.001346,1.000000,0.001346,0.339541
"(LogisticRegression, amazon/chronos-t5-large)",0.603784,1.000000,0.001346,1.000000,0.294070
"(LogisticRegression, ibm-granite/granite-timeseries-patchtsmixer)",0.986698,0.294070,0.339541,0.294070,1.000000


,Dataset,Side,Model,Resampling,Features,Aggregator,Validation,accuracy_score avg,accuracy_score sem,balanced_accuracy_score avg,balanced_accuracy_score sem,matthews_corrcoef avg,matthews_corrcoef sem,f1_score avg,f1_score sem,precision_score avg,precision_score sem,recall_score avg,recall_score sem,roc_auc_score avg,roc_auc_score sem
32,apsync,immersion_10sec,DummyClassifier,None,HandcraftedFeatureExtractor,None,TACV,0.545503,0.080050,0.542930,0.059188,0.071216,0.103792,0.583344,0.163628,0.571770,0.209098,0.705254,0.180745,0.542930,0.059188
34,apsync,immersion_10sec,LogisticRegression,None,AutonLab/MOMENT-1-large,MeanChanAggregator,TACV,0.488051,0.208738,0.526773,0.036324,0.085154,0.108571,0.533131,0.271428,0.559069,0.208522,0.749482,0.350237,0.526773,0.036324
31,apsync,immersion_10sec,LogisticRegression,None,HandcraftedFeatureExtractor,None,TACV,0.311145,0.142635,0.328524,0.131626,-0.355593,0.265080,0.304784,0.219214,0.323593,0.197606,0.375569,0.262681,0.328524,0.131626
30,apsync,immersion_10sec,LogisticRegression,None,amazon/chronos-t5-large,MeanChanAggregator,TACV,0.500288,0.199609,0.514770,0.050704,0.032300,0.159239,0.608019,0.213006,0.547907,0.208574,0.854995,0.235896,0.514770,0.050704
33,apsync,immersion_10sec,LogisticRegression,None,amazon/chronos-t5-small,MeanChanAggregator,TACV,0.452856,0.174762,0.511186,0.051142,0.028248,0.104610,0.533535,0.178035,0.542154,0.203816,0.679710,0.234199,0.511186,0.051142


Friedman Test: statistic=13.800, pvalue=0.003
Nemenyi post-hoc test


...

Friedman Test: statistic=21.000, pvalue=0.000
Nemenyi post-hoc test


,"(DummyClassifier, HandcraftedFeatureExtractor)","(LogisticRegression, AutonLab/MOMENT-1-large)","(LogisticRegression, HandcraftedFeatureExtractor)","(LogisticRegression, amazon/chronos-t5-large)"
"(DummyClassifier, HandcraftedFeatureExtractor)",1.000000,0.468608,0.019635,0.000081
"(LogisticRegression, AutonLab/MOMENT-1-large)",0.468608,1.000000,0.468608,0.019635
"(LogisticRegression, HandcraftedFeatureExtractor)",0.019635,0.468608,1.000000,0.468608
"(LogisticRegression, amazon/chronos-t5-large)",0.000081,0.019635,0.468608,1.000000


,Dataset,Side,Model,Resampling,Features,Aggregator,Validation,accuracy_score avg,accuracy_score sem,balanced_accuracy_score avg,balanced_accuracy_score sem,matthews_corrcoef avg,matthews_corrcoef sem,f1_score avg,f1_score sem,precision_score avg,precision_score sem,recall_score avg,recall_score sem,roc_auc_score avg,roc_auc_score sem
21,usilaughs,right,DummyClassifier,None,HandcraftedFeatureExtractor,None,TACV,0.500000,0.065591,0.497500,0.064466,-0.004713,0.129485,0.456869,0.067827,0.444018,0.067048,0.475000,0.084939,0.497500,0.064466
26,usilaughs,right,DummyClassifier,None,HandcraftedFeatureExtractor,None,TACV,0.522222,0.063931,0.500833,0.053619,0.002060,0.107809,0.287647,0.235803,0.272747,0.227470,0.308333,0.252401,0.500833,0.053619
23,usilaughs,right,LogisticRegression,None,AutonLab/MOMENT-1-large,MeanChanAggregator,TACV,0.600000,0.044305,0.579167,0.041085,0.177598,0.099592,0.462654,0.057802,0.588405,0.098253,0.391667,0.080833,0.579167,0.041085
20,usilaughs,right,LogisticRegression,None,HandcraftedFeatureExtractor,None,TACV,0.740741,0.073333,0.754167,0.065638,0.517757,0.120780,0.753849,0.051885,0.667946,0.085374,0.875000,0.026089,0.754167,0.065638
19,usilaughs,right,LogisticRegression,None,amazon/chronos-t5-large,MeanChanAggregator,TACV,0.777778,0.091299,0.773333,0.093433,0.561735,0.181537,0.740824,0.104231,0.775075,0.106860,0.733333,0.162087,0.773333,0.093433
22,usilaughs,right,LogisticRegression,None,amazon/chronos-t5-small,MeanChanAggregator,TACV,0.785185,0.062009,0.783333,0.069221,0.579000,0.126937,0.751664,0.088778,0.765517,0.069522,0.766667,0.168268,0.783333,0.069221
29,usilaughs,right,LogisticRegression,None,ibm-granite/granite-timeseries-patchtsmixer,MeanChanAggregator,TACV,0.744444,0.090856,0.754167,0.088702,0.509859,0.172999,0.747267,0.083511,0.674165,0.087644,0.841667,0.084134,0.754167,0.088702


Friedman Test: statistic=30.021, pvalue=0.000
Nemenyi post-hoc test


,"(DummyClassifier, HandcraftedFeatureExtractor)","(DummyClassifier, HandcraftedFeatureExtractor)","(LogisticRegression, AutonLab/MOMENT-1-large)","(LogisticRegression, HandcraftedFeatureExtractor)","(LogisticRegression, amazon/chronos-t5-large)","(LogisticRegression, ibm-granite/granite-timeseries-patchtsmixer)"
"(DummyClassifier, HandcraftedFeatureExtractor)",1.000000,1.000000,0.793037,0.013027,0.001601,0.020756
"(DummyClassifier, HandcraftedFeatureExtractor)",1.000000,1.000000,0.793037,0.013027,0.001601,0.020756
"(LogisticRegression, AutonLab/MOMENT-1-large)",0.793037,0.793037,1.000000,0.342180,0.104321,0.428918
"(LogisticRegression, HandcraftedFeatureExtractor)",0.013027,0.013027,0.342180,1.000000,0.992869,0.999992
"(LogisticRegression, amazon/chronos-t5-large)",0.001601,0.001601,0.104321,0.992869,1.000000,0.980292
"(LogisticRegression, ibm-granite/granite-timeseries-patchtsmixer)",0.020756,0.020756,0.428918,0.999992,0.980292,1.000000
